<a href="https://colab.research.google.com/github/varshaelza/Text-Summarization-Categorization/blob/main/CNNBasedSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentence to Salience

In [ ]:
import pandas as pd
t = pd.read_csv('/content/drive/MyDrive/BBC Dataset/LDA Summarization/LDASummariestrain.csv') 

In [ ]:
t

In [ ]:
rawData = {}
rawSummaries = {}
    
# running count variable -- keeps track of the total size
totalSentences = 0 

In [ ]:
def sep(text):
  import nltk
  nltk.download('punkt')
  import re
  text= text.replace("\t"," ")
  text=re.sub(r'\.+', '.',text)
  k=0
  c=0
  p=0
  a_list=[]
  while k<len(text):
    if c==0 and text[k]=='.':
      a_list.append(text[p:k+1])
      p=k+1
    elif c==0 and text[k]=='"':
      c=1
    elif c==1 and text[k]=='"' and text[k-1]=="." :
      a_list.append(text[p:k+1])
      p=k+1
      c=0
    elif c==1 and text[k]=='"' and text[k-2]=="." :
      a_list.append(text[p:k+1])
      p=k+1
      c=0
    elif c==1 and text[k]=='"':
      c=0
    k=k+1
  if p<len(text)-1:
    a_list.append(text[p:len(text)])
  return a_list
  

In [ ]:
def spl(text):
  t=text.split('\n',1)
  if len(t)!=2:
    return ""
  else :
    return t[1]

In [ ]:
j=0
while j<len(t):
  a=sep(t['News'][j])
  a[0]=spl(a[0])
  a[0]=spl(a[0])
  if a[0]=="":
    del a[0]
  i=0
  while i<len(a):
    a[i] = a[i].replace('\n',' ')
    a[i] = a[i].replace('\t',' ')
    a[i]= a[i].replace(r'\.+', ".")
    a[i]= a[i].replace(r'\ +', " ")
    i=i+1

  b=sep(t['Summary'][j])
  b[0]=spl(b[0])
  b[0]=spl(b[0])
  if b[0]=="":
    del b[0]
  i=0
  while i<len(b):
    b[i] = b[i].replace('\n',' ')
    b[i] = b[i].replace('\t',' ')
    b[i]= b[i].replace(r'\.+', ".")
    b[i]= b[i].replace(r'\ +', " ")
    i=i+1
  
  rawData[j]=a
  rawSummaries[j]=b
  j=j+1


In [ ]:
!pip install rouge/requirements.txt
!pip install rouge-score
import numpy as np

def calc_rouge_scores(pred_summaries, gold_summaries, 
                                 keys=['rouge1', 'rouge2'], use_stemmer=True):
    #Calculate rouge scores
    from rouge_score import rouge_scorer
    scorer = rouge_scorer.RougeScorer(keys, use_stemmer= use_stemmer)
    n = len(pred_summaries)
    scores = [scorer.score(pred_summaries[j], gold_summaries[j]) for 
              j in range(n)] 
              
    #create dict
    dict_scores={}                                                            
    for key in keys:
        dict_scores.update({key: {}})
        
    #populate dict    
    for key in keys:
        
        precision_list = [scores[j][key][0] for j in range(len(scores))]
        recall_list = [scores[j][key][1] for j in range(len(scores))]
        f1_list = [scores[j][key][2] for j in range(len(scores))]

        precision = np.mean(precision_list)
        recall = np.mean(recall_list)
        f1 = np.mean(f1_list)
        
        dict_results = {'recall': recall, 'precision': precision, 'f1': f1}
        
        dict_scores[key] = dict_results
        
    return dict_scores

In [ ]:
saliency={}
i=0
while i<len(rawData):
  saliency[i]=[]
  j=0
  while j<len(rawData[i]):
    s=calc_rouge_scores([rawData[i][j]],[t['Summary'][i]])
    saliency[i].append(0.5*s['rouge1']['f1']+s['rouge2']['f1']*0.5) #alpha=0.5
    j=j+1
  i=i+1

In [ ]:
tot=0;
i=0
while i<len(rawData):
  tot=tot+len(rawData[i])
  i=i+1

In [ ]:
import numpy as np
nx3output = np.zeros((tot, 3), dtype=object)

In [ ]:
i=0
c=0
while i<len(rawData):
  j=0
  while j<len(rawData[i]):
    nx3output[c,0]=i
    nx3output[c,1]=rawData[i][j]
    nx3output[c,2]=saliency[i][j]
    j=j+1
    c=c+1
  i=i+1


In [ ]:
import pickle
f = open("/content/drive/MyDrive/BBC Dataset/CNN Summarisation/sentencesToSaliency.pickle", "wb")
pickle.dump(nx3output, f)

In [ ]:
f.close()

In [ ]:
t.to_csv('/content/drive/MyDrive/BBC Dataset/CNN Summarisation/CNNSummariestrain')

Embedding Sentences ith Word2Vec

In [15]:
!cd word2vec/
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

/bin/bash: line 0: cd: word2vec/: No such file or directory
--2021-05-06 14:39:49--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.228.35
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.228.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  32.7MB/s    in 49s     

2021-05-06 14:40:39 (32.0 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [9]:
import pickle

In [10]:
def loadFromPickle(fileName):
    f = open(fileName, "rb")
    data = pickle.load(f)
    f.close()
    return data 

In [11]:
data = loadFromPickle("/content/drive/MyDrive/BBC Dataset/CNN Summarisation/sentencesToSaliency.pickle")

In [12]:
import gensim
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
#def embed_sentences(data, word2vec_limit = 50000 , NUM_WORDS=20000):

In [13]:
def embed_sentences(data, word2vec_limit = 50000 , NUM_WORDS=20000):   
    '''
    Embed sentences
    Params:
        data             - np.array  [ doc id, sentences, saliency score ]
                            
        word2vec_limit   - int: number of words used in the word embedding provided by Google
                            - ex: 50000
        NUM_WORDS        - int: The maximum number of words to keep, based on word frequency. Only the most common num_words words will be kept.
                            - ex: 20000
       
    Returns:
        input_output        - np.array [embedding matrix , saliency score]
                        
    '''
    
    #setences ex: ["It's the first sentence!","it is the second sentence"]
    sentences = data[:,1]
    
    #Load Google pre-trained words as a model
    embedding_model = gensim.models.KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=word2vec_limit)
    #Convert the model as a dictionnary word_vectors["hello"] will return a vector like [0.3, 3, ... , -4]
    word_vectors = embedding_model
    #print("Embedding for 'hello': ", word_vectors["hello"], "\n")
    
    # Tokenize the sentences, that is to say convert the 2 sentences ["It's the first sentence!","It is the second sentence"] to 2 sequences [[1 4 2 5 3],[1 6 2 7 3]]
    # It handles the ennoying cases (punctuation, Upper cases, etc...)
    tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',lower=True)
    tokenizer.fit_on_texts(sentences)
    sequences = tokenizer.texts_to_sequences(sentences)
    padded_sequences = pad_sequences(sequences)
    #print("Padded_sequences: ", padded_sequences, "\n")   
    
    #Produce a dictionnary mapping words to tokens e.g. {'it': 1, 'the': 2, 'sentence': 3, 's': 4, 'first': 5, 'is': 6, 'second': 7}
    word_index = tokenizer.word_index
    #print("word_index: " , word_index , "\n" )
    
    #Build a dictionnary mapping tokens to vectors e.g. {'1': [2, ... , -3] ; '2': [4, ... , 0.8] ; ... }
    embedding_weights = {key: embedding_model[word] if word in word_vectors.vocab else
                              np.random.uniform(-0.25, 0.25, word_vectors.vector_size)
                        for word, key in word_index.items()}
    # Add the token "0", used for padding
    embedding_weights[0] = np.zeros(word_vectors.vector_size)
   
    #print("Embedding weights: " , embedding_weights , "\n")
    
    #Build a 3D array: 1D for the sentences, 1D for the words and 1D for the word2vec dimensions. 
    embedded_sentences = np.stack([np.stack([embedding_weights[token] for token in sentence]) for sentence in padded_sequences])
    
    #Add back the saliency scores
    #input_output = np.column_stack((embedded_sentences,data[:,2]))    
    
    input_output = np.array([])
    for i in range(len(data)):
        input_output = np.append(input_output,np.array([ embedded_sentences[i] , data[i,2] ]) )
        
    del embedding_model
    
    return input_output

In [6]:
len(data)

31592

In [7]:
data1=data[0:3000]
data1=embed_sentences(data1)
f = open('/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency1.pickle', "wb")
pickle.dump(data1,f)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [6]:
data1=data[3000:4000]
data1=embed_sentences(data1)
f = open('embeddingsToSaliency2.pickle', "wb")
pickle.dump(data1,f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [7]:
data1=data[4000:5000]
data1=embed_sentences(data1)
f = open('embeddingsToSaliency2a.pickle', "wb")
pickle.dump(data1,f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [8]:
data1=data[5000:6000]
data1=embed_sentences(data1)
f = open('embeddingsToSaliency2b.pickle', "wb")
pickle.dump(data1,f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [8]:
data1=data[6000:9000]
data1=embed_sentences(data1)
f = open('/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency3.pickle', "wb")
pickle.dump(data1,f)
 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [11]:
data1=data[9000:12000]
data1=embed_sentences(data1)
f = open('/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency4.pickle', "wb")
pickle.dump(data1,f)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [12]:
data1=data[12000:15000]
data1=embed_sentences(data1)
f = open('/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency5.pickle', "wb")
pickle.dump(data1,f)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [16]:
data1=data[15000:17000]
data1=embed_sentences(data1)
f = open('embeddingsToSaliency6.pickle', "wb")
pickle.dump(data1,f)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [17]:
data1=data[17000:18000]
data1=embed_sentences(data1)
f = open('embeddingsToSaliency6a.pickle', "wb")
pickle.dump(data1,f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [6]:
data1=data[18000:21000]
data1=embed_sentences(data1)
f = open('/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency7.pickle', "wb")
pickle.dump(data1,f)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [7]:
data1=data[21000:24000]
data1=embed_sentences(data1)
f = open('/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency8.pickle', "wb")
pickle.dump(data1,f)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [8]:
data1=data[24000:27000]
data1=embed_sentences(data1)
f = open('/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency9.pickle', "wb")
pickle.dump(data1,f)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [14]:
data1=data[27000:29000]
data1=embed_sentences(data1)
f = open('embeddingsToSaliency10.pickle', "wb")
pickle.dump(data1,f)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [15]:
data1=data[29000:30000]
data1=embed_sentences(data1)
f = open('embeddingsToSaliency10a.pickle', "wb")
pickle.dump(data1,f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [16]:
data1=data[30000:len(data)]
data1=embed_sentences(data1)
f = open('embeddingsToSaliency11.pickle', "wb")
pickle.dump(data1,f)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
#2 2a 2b 6 6a 10 10a 11